In [10]:
from langchain.agents import Tool
from langchain import HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import os 

def read_token_from_file(file_path="Token.env"):
    with open(file_path, "r") as file:
        return file.read().strip()

HUGGINGFACEHUB_API_TOKEN = read_token_from_file("Token.env")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Load Hugging Face model and tokenizer
model_name = "microsoft/codebert-base"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up Hugging Face components
pipeline = HuggingFacePipeline(model=model_name, api_token=HUGGINGFACEHUB_API_TOKEN)

# Define intents, entities, and agents
intents = ["code_suggestion", "error_explanation", "documentation_lookup"]
entities = ["code_snippet", "error_message", "api_name"]

@Tool
def code_suggestion_agent(prompt):
    completions = pipeline(prompt)
    return completions

@Tool
def error_explanation_agent(error_message):
    explanation = pipeline(f"Explain this error: {error_message}")
    return explanation

@Tool
def documentation_agent(api_name):
    docs = pipeline(f"Find documentation for {api_name}")
    return docs

# Register agents
manager = HuggingFaceHub()
manager.register_tool(code_suggestion_agent, intents=["code_suggestion"])
manager.register_tool(error_explanation_agent, intents=["error_explanation"])
manager.register_tool(documentation_agent, intents=["documentation_lookup"])

# Conversation loop
while True:
    user_input = input("You: ")
    intent, entities = manager.classify_intent(user_input)
    response = manager.execute(intent, entities)
    print("Chatbot: ", response)


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValidationError: 2 validation errors for HuggingFacePipeline
api_token
  extra fields not permitted (type=value_error.extra)
model
  extra fields not permitted (type=value_error.extra)